<a href="https://colab.research.google.com/github/cwangf00d/SkinDictionary/blob/main/SkinDict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All Web Scraping/Data Cleaning for Skin Dictionary

Hello friends!

In order to gather the data necessary for Skin Dictionary, I scraped information from multiple sources:
- [Byrdie's Skincare Ingredients A-Z](https://www.byrdie.com/skincare-ingredients-glossary-4800556)
- [The Ordinary's Catalog](https://theordinary.deciem.com/)
- [Sephora's Best-Selling Skincare Catalog](https://www.sephora.com/shop/skin-care-solutions?pageSize=300)

Built with the following packages
- Beautiful Soup from bs4
- requests
- re
- JSON
- Numpy
- Pandas
- Matplotlib

For reference, I used 
- [HODP's Guide to Web Scraping](https://docs.hodp.org/docs/scraping)
- [W3 Schools](https://www.w3schools.com/)
- [GoTrained Python Tutorials](https://python.gotrained.com/beautifulsoup-extracting-urls/)
- [Python Spot](https://pythonspot.com/extract-links-from-webpage-beautifulsoup/)
- [Kite](https://www.kite.com/python/examples/1730/beautifulsoup-find-the-next-sibling-of-a-tag)
- [Crummy](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)
- [Link Gopher](https://sites.google.com/site/linkgopher/)


## Scraping
I first went through and tried to scrape each of my three websites:
- Byrdie
- The Ordinary
- Sephora

### Scraping Byrdie
Byrdie's A-Z ingredients contained all of the necessary information I wanted to use as my preliminary source for various chemicals used in skincare products and what skin conditions/ailments they were best for treating.

I decided to first try and scrape one page, which we can see with the **allantoin** example.

In [72]:
# Importing in necessary packages
from bs4 import BeautifulSoup
import requests


In [73]:
# Scraping the page for allantoin using the exact URL for the page from Byrdie on allantoin
my_page = requests.get('https://www.byrdie.com/allantoin-for-skin-4769521', auth=('user', 'pass'))
allantoin_soup = BeautifulSoup(my_page.content)
print(allantoin_soup.prettify()[:])

<!DOCTYPE html>
<html class="comp structuredContentTemplate article-layout base-layout mntl-html" data-ab="54,99,56,52" data-beauty-resource-version="1.104.0" data-mantle-resource-version="3.12.216" data-resource-version="1.104.0" data-tracking-container="true" id="structuredContentTemplate_1-0" lang="en">
 <!--
<globe-environment environment="prod" application="beauty" dataCenter="us-east-1"/>
-->
 <head class="loc head">
  <script type="text/javascript">
  </script>
  <link href="//js-sec.indexww.com" rel="preconnect"/>
  <link href="//c.amazon-adsystem.com" rel="preconnect"/>
  <link href="//cdn.adsafeprotected.com" rel="preconnect"/>
  <link href="//securepubads.g.doubleclick.net" rel="preconnect"/>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="max-image-preview:large, NOODP, NOYDIR" name="robots"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://www.byrdie.com/allantoin-for-skin-4

Upon looking into the formatting of Byrdie's ingredient information pages, I quickly realized that it would be impractical to scrape the entire webpage. To give myself a starting point, I decided to scrape starting from their fast facts boxes.

In [74]:
# these are the categories that byrdie labeled their fast facts boxes with (slightly altered titles)
categories = ['type', 'benefits', 'target', 'frequency', 'compatible', 'incompatible']



In [75]:
# function to scrape webpages for essential info in fast facts boxes
def parse_store(ingredient_soup):
  #final list returning containing one dictionary of all of the features as keys and their corresponding text as outputs
  featureList = []
  features = ingredient_soup.find_all('div', {'class':'comp theme-ffingredients mntl-sc-block beauty-sc-block-callout--ffingredients beauty-sc-block-callout mntl-sc-block-callout mntl-block'})
  for feature in features:
    entry = {}
    for p in feature.find_all('p', {'class': 'comp mntl-sc-block-callout-heading mntl-text-block'}):
      entry['name'] = p.text.replace('\n', "")
    for body in feature.find_all('div', {'class' : 'comp mntl-sc-block-callout-body mntl-text-block'}):
      texts = body.find_all('p')
      if len(texts) == 0:
        texts = body.find_all('li')
      counter = 0
      for words in texts:
        entry[categories[counter]] = words.text.strip()
        if (":" in entry[categories[counter]]):
          entry[categories[counter]] = entry[categories[counter]].split(":")[1].strip()
        counter +=1
    featureList.append(entry)
  return featureList

In [76]:
# using parse_store function to scrape allantoin
allantoin_features = parse_store(allantoin_soup)
print(allantoin_features)

[{'name': 'Allantoin', 'type': 'Moisturizer and exfoliator.', 'benefits': 'Moisturizes, soothes itchiness, and heals injured skin.', 'target': 'In general, those with dry, itchy skin looking for better skin texture and smoothness, regardless of skin type, age, or conditions.', 'frequency': 'Can be used day in and day out without general side\xa0effects of any kind, according to Shamban.', 'compatible': 'Herrmann says when used in scar creams, allantoin is often combined with ingredients like silicone and other emollients to amplify results.', 'incompatible': 'Herrmann adds that combining it with other exfoliants could produce an exaggerated reaction, causing excess irritation, if strengths are too high.'}]


Now that we have a function that can take a particular page's url and then extract all of the information we want to collect from it, I wanted to scrape all of the urls from their index page to be able to loop through the urls.

In [77]:
# from documentation I found online about how to extract urls using beautiful soup
import re

In [78]:
# function to loop through the index page and gather all of the urls on the Byrdie Index page specifically
def parse_index_urls(ingredient_soup):
  urlList = []
  blocks = ingredient_soup.find_all('div', {'class' : 'comp l-container alphabetical-list'})
  for block in blocks:
    for link in block.find_all('li', {'class' : 'alphabetical-list__sublist-item'}):
      urlList.append(link.find('a', attrs = {'href': re.compile("^https://")}).get('href'))
  return urlList

In [79]:
# getting the urls specifically from byrdie's index for their ingredients glossary
index = requests.get('https://www.byrdie.com/skincare-ingredients-glossary-4800556', auth=('user', 'pass'))
index_soup = BeautifulSoup(index.content)
links = parse_index_urls(index_soup)
print(links)

['https://www.byrdie.com/allantoin-for-skin-4769521', 'https://www.byrdie.com/aloe-vera-for-skin-4777483', 'https://www.byrdie.com/alpha-hydroxy-acid-for-skin-4844378', 'https://www.byrdie.com/amino-acids-for-skin', 'https://www.byrdie.com/amoxicillin-for-acne-4843079', 'https://www.byrdie.com/apple-vinegar-cider-for-skin', 'https://www.byrdie.com/arbutin-serum', 'https://www.byrdie.com/argireline-for-skin-4771349', 'https://www.byrdie.com/ascorbyl-glucoside-for-skin-4773538', 'https://www.byrdie.com/avocado-oil-for-skin-4846442', 'https://www.byrdie.com/azelaic-acid-4705426', 'https://www.byrdie.com/what-is-bakuchiol', 'https://www.byrdie.com/benzoyl-peroxide-acne-4771462', 'https://www.byrdie.com/benzyl-alcohol-for-skin-4774552', 'https://www.byrdie.com/beta-glucan-skincare', 'https://www.byrdie.com/biotin-for-skin-4801191', 'https://www.byrdie.com/butylene-glycol-for-skin-4774849', 'https://www.byrdie.com/caffeine-in-skincare', 'https://www.byrdie.com/caprylyl-glycol-4774428', 'http

In [80]:
# iterating through list of links to scrape from all pages
# ingredients will be a list of dictionaries where each dictionary has the feature categories as keys and the text as values.
ingredients = []
for ingredient in links:
  curr_index = requests.get(ingredient, auth=('user', 'pass'))
  curr_soup = BeautifulSoup(curr_index.content)
  curr_list = parse_store(curr_soup)
  for item in curr_list:
    ingredients.append(item)
print(ingredients)


KeyboardInterrupt: ignored

Now that we have scraped everything into a list of dictionaries, we want to import it into a more usable format. To do so, we will first convert it to a json and then to a pandas dataframe with which we can clean.

In [ ]:
# creating the json
import json
with open('byrdie.json', 'w') as fout:
  json.dump(ingredients, fout)

In [ ]:
# opening the json we created
with open('byrdie.json', 'r') as fd:
  ingredients = json.load(fd)
print(ingredients)

In [ ]:
# importing in extra packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
# converting the json to a pandas dataframe
byrdie_df = pd.DataFrame(ingredients)

# optimizing dataframe for my viewing
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

byrdie_df

### Scraping The Ordinary

For the first iteration of my website, I started with a smaller set of products, so I decided to first try scraping The Ordinary's Catalog of 55 skincare products.

As per usual, I started with just one page, a page on this vitamin C product.

In [ ]:
# scraping just one vitamin C product page from the Ordinary's website
my_page = requests.get('https://theordinary.deciem.com/product/rdn-ascorbic-acid-8pct-alpha-arbutin-2pct-30ml?redir=1', auth=('user', 'pass'))
vitc_soup = BeautifulSoup(my_page.content)
print(vitc_soup.prettify()[:])

I wanted to grab the name, price, ingredients, and rating but the rating proved to be too elusive for Beautiful Soup to catch. Online sources suggested Selenium, but I was unable to get it to work either so I decided to forego scraping the ratings.

In [ ]:
# function to get name, price, ingredients from product page on The Ordinary
def parse_theord_store(ingredient_soup, link):
  featureList = []
  features = ingredient_soup.find_all('div', {'class':'content-outer'})
  for feature in features:
    #name, price, ingredients
    entry = {}
    # getting name, ingredients
    for p in feature.find_all('div', {'class' : 'p'}):
      entry['name'] = p.find('h1').text
      entry['link'] = link
      for h4 in p.find_all('h4'):
        if h4.text == "Ingredients":
          entry['ingredients'] = h4.next_sibling.next_sibling.text.replace("\n ", "").strip()
    # getting price
    for target in feature.find_all('div', {'class': 'col-image'}):
      entry['price'] = float(target.find('span', {'class': 'price org'}).text.replace(" USD", ''))
    featureList.append(entry)
  return featureList

In [ ]:
# testing my function on that one vitamin C product
vit_c_things = parse_theord_store(vitc_soup, "https://theordinary.deciem.com/product/rdn-ascorbic-acid-8pct-alpha-arbutin-2pct-30ml?redir=1")
print(vit_c_things)

The method that had worked really well for scraping URLs from Byrdie did not work for the Ordinary, so I just scraped the links by hand.

In [ ]:
the_ord_links = ["https://theordinary.deciem.com/product/rdn-euk-134-01pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-pycnogenol-5pct-15ml?redir=1", "https://theordinary.deciem.com/product/rdn-resveratrol-3pct-ferulic-acid-3pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-squalane-cleanser-50ml?redir=1", "https://theordinary.deciem.com/product/rdn-coverage-foundation-10-n-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-serum-foundation-10-n-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-dropper-1pc?redir=1", "https://theordinary.deciem.com/product/rdn-high-adherence-silicone-primer-30ml#siliconlink", "https://theordinary.deciem.com/product/rdn-high-spreadability-fluid-primer-30ml#fluidlink", "https://theordinary.deciem.com/product/rdn-aha-30pct-bha-2pct-peeling-solution-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-azelaic-acid-suspension-10pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-glycolic-acid-7pct-toning-solution-240ml?redir=1", "https://theordinary.deciem.com/product/rdn-lactic-acid-10pct-ha-2pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-lactic-acid-5pct-ha-2pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-mandelic-acid-10pct-ha-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-salicylic-acid-2pct-masque-50ml?redir=1", "https://theordinary.deciem.com/product/rdn-salicylic-acid-2pct-solution-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-multi-peptide-serum-for-hair-density-60ml?redir=1", "https://theordinary.deciem.com/product/rdn-b-oil-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-100pct-cold-pressed-virgin-marula-oil-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-100pct-organic-cold-pressed-borage-seed-oil-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-100pct-organic-cold-pressed-moroccan-argan-oil-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-100pct-organic-cold-pressed-rose-hip-seed-oil-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-100pct-organic-virgin-chia-seed-oil-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-100pct-organic-virgin-sea-buckthorn-fruit-oil-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-100pct-plant-derived-hemi-squalane-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-100pct-plant-derived-squalane-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-amino-acids-b5-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-hyaluronic-acid-2pct-b5-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-marine-hyaluronics-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-natural-moisturizing-factors-ha-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-100pct-niacinamide-powder-20g?redir=1", "https://theordinary.deciem.com/product/rdn-alpha-arbutin-2pct-ha-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-caffeine-solution-5pct-egcg-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-niacinamide-10pct-zinc-1pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-buffet-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-buffet-copper-peptides-1pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-argireline-solution-10pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-matrixyl-10pct-ha-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-advanced-retinoid-2pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-granactive-retinoid-2pct-in-squalane-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-granactive-retinoid-5pct-in-squalane-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-retinol-02pct-in-squalane-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-retinol-05pct-in-squalane-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-retinol-1pct-in-squalane-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-mineral-uv-filters-spf-15-with-antioxidants-50ml?redir=1", "https://theordinary.deciem.com/product/rdn-mineral-uv-filters-spf-30-with-antioxidants-50ml?redir=1", "https://theordinary.deciem.com/product/rdn-100pct-l-ascorbic-acid-powder-20g?redir=1", "https://theordinary.deciem.com/product/rdn-ascorbic-acid-8pct-alpha-arbutin-2pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-ascorbyl-glucoside-solution-12pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-ascorbyl-glucoside-solution-12pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-ascorbyl-tetraisopalmitate-solution-20pct-in-vitamin-f-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-ethylated-ascorbic-acid-15pct-solution-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-magnesium-ascorbyl-phosphate-10pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-vitamin-c-suspension-23pct-ha-spheres-2pct-30ml?redir=1", "https://theordinary.deciem.com/product/rdn-vitamin-c-suspension-30pct-in-silicone-30ml?redir=1"]
the_ord_links

In [ ]:
# iterating through url list to scrape from all pages
the_ord_products = []
for product in the_ord_links:
  curr_index = requests.get(product, auth=('user', 'pass'))
  curr_soup = BeautifulSoup(curr_index.content)
  curr_list = parse_theord_store(curr_soup, product)
  for item in curr_list:
    the_ord_products.append(item)
print(the_ord_products)

Transferring to JSON -> pandas dataframe

In [ ]:
# dumping to JSON
with open('the_ordinary.json', 'w') as fout:
  json.dump(the_ord_products, fout)

In [ ]:
# reading from JSON
with open('the_ordinary.json', 'r') as fd:
  the_ord_products = json.load(fd)
print(the_ord_products)

In [ ]:
# converting to pandas dataframe
the_ordinary_df = pd.DataFrame(the_ord_products)
the_ordinary_df = the_ordinary_df.dropna()
the_ordinary_df

### Scraping Sephora
After I got my whole website to work with just the 55 products from The Ordinary, I decided that I wanted to try scraping Sephora instead. This is because the majority of the products from The Ordinary were in Sephora's catalog and that way I'd be able to get around 300 products with more information about their popularity and brand as well.

As per usual, I started with one page.

In [ ]:
# scraping this page on this hydration cream
my_page = requests.get('https://www.sephora.com/product/ultra-repair-cream-intense-hydration-P248407?skuId=1784826&icid2=products%20grid:p248407', auth=('user', 'pass'))
cream_soup = BeautifulSoup(my_page.content)
print(cream_soup.prettify()[:])

From Sephora, I was able to scrape name, brand, link, price, popularity (loves).

In [ ]:
# function for scraping sephora for name, brand, link, price, popularity given a link for that specific product
def parse_seph_store(ingredient_soup, link):
  featureList = []
  # WANT TO GRAB: name, brand, link, price, loves, short-description, skin type, image
  features = ingredient_soup.find_all('div', {'class':'css-1kk8dps e65zztl0'})
  for feature in features:
    entry = {}
    # getting name, brand, loves, link
    for div in feature.find_all('div', {'class' : 'css-1xy0oe1 e65zztl0'}):
      entry['name'] = div.find('a').next_sibling.text
      entry['brand'] = div.find('span', {'class': 'css-57kn72'}).text
      entry['loves'] = int(div.find('span', {'data-at': 'product_love_count'}).text)
      entry['link'] = link
    #getting price
    for div in feature.find_all('div', {'class': 'css-1ydibbt e65zztl0'}):
      entry['price'] = float(div.find('span', {'data-at':'price'}).text.replace("$", ""))  
    # get ingredients
    for div in feature.find_all('div', {'id': 'tabpanel2'}):
      ingredients = div.text.split(".")
      entry['ingredients'] = ingredients[len(ingredients) - 2].strip()
    featureList.append(entry)
  return featureList

In [ ]:
# testing out my function on this first cream
cream_features = parse_store(cream_soup, "https://www.sephora.com/product/ultra-repair-cream-intense-hydration-P248407?skuId=1784826&icid2=products%20grid:p248407")
print(cream_features)

Getting urls from Sephora, I ended up using LinkGopher, a chrome extension, because I was once again having trouble with actually scraping it.

In [ ]:
# taking urls from sephora extracted with link gopher
my_str = '''https://www.sephora.com/product/100-percent-pure-argan-oil-P218700?icid2=products grid:p218700
https://www.sephora.com/product/100-pure-argan-oil-light-P380805?icid2=products grid:p380805
https://www.sephora.com/product/3-little-wonders-P124716?icid2=products grid:p124716
https://www.sephora.com/product/5-in-1-face-cream-spf-30-P263006?icid2=products grid:p263006
https://www.sephora.com/product/7-day-scrub-cream-rinse-off-formula-P122661?icid2=products grid:p122661
https://www.sephora.com/product/a-passioni-retinol-cream-P439926?icid2=products grid:p439926
https://www.sephora.com/product/a-perfect-world-spf-40-age-defense-moisturizer-with-white-tea-P417307?icid2=products grid:p417307
https://www.sephora.com/product/abeille-royale-youth-watery-oil-P422895?icid2=products grid:p422895
https://www.sephora.com/product/acne-healing-dots-P421275?icid2=products grid:p421275
https://www.sephora.com/product/acne-solutions-clarifying-lotion-P188307?icid2=products grid:p188307
https://www.sephora.com/product/acne-solutions-cleansing-face-body-soap-P196524?icid2=products grid:p196524
https://www.sephora.com/product/acne-solutions-cleansing-foam-P188309?icid2=products grid:p188309
https://www.sephora.com/product/acne-solutions-clinical-clearing-gel-P385432?icid2=products grid:p385432
https://www.sephora.com/product/acne-solutions-clinical-clearing-kit-P387942?icid2=products grid:p387942
https://www.sephora.com/product/active-moist-P423142?icid2=products grid:p423142
https://www.sephora.com/product/advanced-genifique-youth-activating-serum-P379510?icid2=products grid:p379510
https://www.sephora.com/product/age-arrest-anti-wrinkle-cream-P382356?icid2=products grid:p382356
https://www.sephora.com/product/aha-30-bha-2-peeling-solution-P442563?icid2=products grid:p442563
https://www.sephora.com/product/aha-bha-exfoliating-cleanser-P4010?icid2=products grid:p4010
https://www.sephora.com/product/all-about-eyes-P122774?icid2=products grid:p122774
https://www.sephora.com/product/all-about-eyes-rich-P174502?icid2=products grid:p174502
https://www.sephora.com/product/all-day-hydrator-hydrate-glow-P438617?icid2=products grid:p438617
https://www.sephora.com/product/alpha-beta-peel-extra-strength-daily-peel-P269122?icid2=products grid:p269122
https://www.sephora.com/product/alpha-beta-ultra-gentle-daily-peel-sensitive-skin-P398761?icid2=products grid:p398761
https://www.sephora.com/product/alpha-beta-universal-daily-peel-P377533?icid2=products grid:p377533
https://www.sephora.com/product/anti-aging-cleansing-gel-P39274?icid2=products grid:p39274
https://www.sephora.com/product/avocado-melt-retinol-eye-sleeping-mask-P447791?icid2=products grid:p447791
https://www.sephora.com/product/avocado-melt-retinol-sleeping-mask-P449160?icid2=products grid:p449160
https://www.sephora.com/product/b-hydra-intensive-hydration-gel-P406712?icid2=products grid:p406712
https://www.sephora.com/product/balance-it-all-essentials-set-P416954?icid2=products grid:p416954
https://www.sephora.com/product/balancing-force-oil-control-toner-P416816?icid2=products grid:p416816
https://www.sephora.com/product/beauty-elixir-P6025?icid2=products grid:p6025
https://www.sephora.com/product/benefiance-wrinkle-smoothing-cream-enriched-P448202?icid2=products grid:p448202
https://www.sephora.com/product/benefiance-wrinkle-smoothing-day-cream-broad-spectrum-spf-23-P448201?icid2=products grid:p448201
https://www.sephora.com/product/benefiance-wrinkle-smoothing-eye-cream-P448203?icid2=products grid:p448203
https://www.sephora.com/product/beste-no-9-jelly-cleanser-P426836?icid2=products grid:p426836
https://www.sephora.com/product/beta-hydroxy-acid-P442871?icid2=products grid:p442871
https://www.sephora.com/product/bienfait-multi-vital-night-high-potency-night-moisturizing-cream-vita-nutri-8-P185818?icid2=products grid:p185818
https://www.sephora.com/product/bienfait-multi-vital-spf-30-cream-high-potency-vitamin-enriched-daily-moisturizing-cream-P158043?icid2=products grid:p158043
https://www.sephora.com/product/bio-performance-advanced-super-revitalizing-cream-P173652?icid2=products grid:p173652
https://www.sephora.com/product/biossance-squalane-10-lactic-acid-resurfacing-night-serum-P453224?icid2=products grid:p453224
https://www.sephora.com/product/black-tea-age-delay-eye-concentrate-P384778?icid2=products grid:p384778
https://www.sephora.com/product/black-tea-firming-corset-cream-P415107?icid2=products grid:p415107
https://www.sephora.com/product/black-tea-firming-overnight-mask-P391799?icid2=products grid:p391799
https://www.sephora.com/product/black-tea-instant-perfecting-mask-P217513?icid2=products grid:p217513
https://www.sephora.com/product/black-tea-kombucha-facial-treatment-essence-P428628?icid2=products grid:p428628
https://www.sephora.com/product/blue-tansy-aha-bha-resurfacing-clarity-mask-P400207?icid2=products grid:p400207
https://www.sephora.com/product/bye-bye-breakout-full-coverage-treatment-concealer-P427736?icid2=products grid:p427736
https://www.sephora.com/product/c-collagen-brighten-firm-vitamin-c-serum-P415209?icid2=products grid:p415209
https://www.sephora.com/product/c-e-o-c-e-antioxidant-protect-repair-moisturizer-P416139?icid2=products grid:p416139
https://www.sephora.com/product/c-firma-day-serum-P400259?icid2=products grid:p400259
https://www.sephora.com/product/c-tango-multivitamin-eye-cream-P429515?icid2=products grid:p429515
https://www.sephora.com/product/caffeine-eye-cream-P443840?icid2=products grid:p443840
https://www.sephora.com/product/capture-totale-dreamskin-advanced-P415619?icid2=products grid:p415619
https://www.sephora.com/product/caudalie-vinoperfect-brightening-moisturizer-with-niacinamide-P456614?icid2=products grid:p456614
https://www.sephora.com/product/ceramic-slip-clay-cleanser-P309310?icid2=products grid:p309310
https://www.sephora.com/product/cicapair-tiger-grass-camo-drops-P429250?icid2=products grid:p429250
https://www.sephora.com/product/city-block-purifying-charcoal-clay-mask-scrub-P409919?icid2=products grid:p409919
https://www.sephora.com/product/clarifying-cleanser-P4016?icid2=products grid:p4016
https://www.sephora.com/product/clarifying-lotion-2-P122882?icid2=products grid:p122882
https://www.sephora.com/product/clarifying-lotion-3-P122881?icid2=products grid:p122881
https://www.sephora.com/product/clear-improvement-active-charcoal-mask-to-clear-pores-P297524?icid2=products grid:p297524
https://www.sephora.com/product/clearly-corrective-dark-spot-solution-P422022?icid2=products grid:p422022
https://www.sephora.com/product/clinique-smart-custom-repair-serum-P387616?icid2=products grid:p387616
https://www.sephora.com/product/coconut-crush-scrub-P413967?icid2=products grid:p413967
https://www.sephora.com/product/coconut-lip-love-P416145?icid2=products grid:p416145
https://www.sephora.com/product/coconut-melt-P413350?icid2=products grid:p413350
https://www.sephora.com/product/coconut-water-cream-P426707?icid2=products grid:p426707
https://www.sephora.com/product/collagen-serum-P443846?icid2=products grid:p443846
https://www.sephora.com/product/complete-eye-renewal-balm-P282938?icid2=products grid:p282938
https://www.sephora.com/product/counter-balance-oil-control-hydrator-P416818?icid2=products grid:p416818
https://www.sephora.com/product/creme-ancienne-P42592?icid2=products grid:p42592
https://www.sephora.com/product/d-bronzi-tm-anti-pollution-sunshine-serum-P432668?icid2=products grid:p432668
https://www.sephora.com/product/daily-cleanse-supplements-P386754?icid2=products grid:p386754
https://www.sephora.com/product/daily-microfoliant-P423688?icid2=products grid:p423688
https://www.sephora.com/product/dark-spots-P442536?icid2=products grid:p442536
https://www.sephora.com/product/deep-cleansing-exfoliator-P374574?icid2=products grid:p374574
https://www.sephora.com/product/deep-dive-glycolic-facial-cleanser-P385674?icid2=products grid:p385674
https://www.sephora.com/product/dermapore-ultrasonic-pore-extractor-serum-infuser-P448181?icid2=products grid:p448181
https://www.sephora.com/product/dermask-intra-jet-wrinkless-solution-P397626?icid2=products grid:p397626
https://www.sephora.com/product/dermask-micro-jet-clearing-solution-P397624?icid2=products grid:p397624
https://www.sephora.com/product/dermask-water-jet-vital-hydra-solution-P397623?icid2=products grid:p397623
https://www.sephora.com/product/detoxifying-black-cleanser-P302940?icid2=products grid:p302940
https://www.sephora.com/product/double-duty-face-moisturizer-broad-spectrum-spf-20-P40629?icid2=products grid:p40629
https://www.sephora.com/product/dr-barbara-sturm-face-cream-P447781?icid2=products grid:p447781
https://www.sephora.com/product/dr-barbara-sturm-face-cream-rich-P448553?icid2=products grid:p448553
https://www.sephora.com/product/dr-barbara-sturm-hyaluronic-serum-P447212?icid2=products grid:p447212
https://www.sephora.com/product/dr-barbara-sturm-super-anti-aging-serum-P447787?icid2=products grid:p447787
https://www.sephora.com/product/dramatically-different-moisturizing-gel-P122900?icid2=products grid:p122900
https://www.sephora.com/product/dramatically-different-moisturizing-lotion-P381030?icid2=products grid:p381030
https://www.sephora.com/product/drunk-elephant-been-long-day-P458208?icid2=products grid:p458208
https://www.sephora.com/product/drunk-elephant-lala-retro-whipped-moisturizer-with-ceramides-P446938?icid2=products grid:p446938
https://www.sephora.com/product/drunk-elephant-time-to-wake-up-P458207?icid2=products grid:p458207
https://www.sephora.com/product/dynamic-skin-recovery-spf50-P443354?icid2=products grid:p443354
https://www.sephora.com/product/emerald-cbd-adaptogens-deep-moisture-glow-oil-P442564?icid2=products grid:p442564
https://www.sephora.com/product/eradikate-acne-treatment-P232903?icid2=products grid:p232903
https://www.sephora.com/product/eradikate-daily-cleanser-acne-treatment-P415667?icid2=products grid:p415667
https://www.sephora.com/product/espada-acne-clearing-blue-light-pen-P419633?icid2=products grid:p419633
https://www.sephora.com/product/essential-c-day-moisture-spf-30-broad-spectrum-spf-30-pa-P9940?icid2=products grid:p9940
https://www.sephora.com/product/eudermine-revitalizing-essence-P173653?icid2=products grid:p173653
https://www.sephora.com/product/exfoliating-peel-gel-P393292?icid2=products grid:p393292
https://www.sephora.com/product/exfolikate-cleanser-daily-foaming-wash-P404322?icid2=products grid:p404322
https://www.sephora.com/product/exfolikate-intensive-exfoliating-treatment-P232915?skuId=1201755&icid2=products grid:p232915
https://www.sephora.com/product/face-buff-energizing-scrub-P12571?icid2=products grid:p12571
https://www.sephora.com/product/face-finishing-moisturizer-P211304?icid2=products grid:p211304
https://www.sephora.com/product/facial-radiance-pads-P278317?icid2=products grid:p278317
https://www.sephora.com/product/facial-treatment-essence-P375849?icid2=products grid:p375849
https://www.sephora.com/product/facial-treatment-mask-P375852?icid2=products grid:p375852
https://www.sephora.com/product/ferulic-acid-retinol-brightening-solution-P375269?icid2=products grid:p375269
https://www.sephora.com/product/ferulic-retinol-anti-aging-moisturizer-P384536?icid2=products grid:p384536
https://www.sephora.com/product/ferulic-retinol-eye-cream-P388622?icid2=products grid:p388622
https://www.sephora.com/product/ferulic-retinol-triple-correction-eye-serum-P377531?icid2=products grid:p377531
https://www.sephora.com/product/ferulic-retinol-wrinkle-recovery-overnight-serum-P404486?icid2=products grid:p404486
https://www.sephora.com/product/ferulic-retinol-wrinkle-recovery-peel-P404233?icid2=products grid:p404233
https://www.sephora.com/product/find-your-balance-oil-control-cleanser-P416815?icid2=products grid:p416815
https://www.sephora.com/product/firmx-peeling-gel-P377561?icid2=products grid:p377561
https://www.sephora.com/product/first-care-activating-serum-P448915?icid2=products grid:p448915
https://www.sephora.com/product/genifique-eye-youth-activating-eye-concentrate-P247905?icid2=products grid:p247905
https://www.sephora.com/product/genius-liquid-collagen-P421277?icid2=products grid:p421277
https://www.sephora.com/product/genius-ultimate-anti-aging-cream-P384537?icid2=products grid:p384537
https://www.sephora.com/product/genoptics-aura-essence-P410756?icid2=products grid:p410756
https://www.sephora.com/product/ginzing-refreshing-scrub-cleanser-P386197?icid2=products grid:p386197
https://www.sephora.com/product/glow-recipe-banana-souffle-moisture-cream-P453223?icid2=products grid:p453223
https://www.sephora.com/product/glow-recipe-plum-plump-hyaluronic-acid-serum-P462699?icid2=products grid:p462699
https://www.sephora.com/product/glow-recipe-watermelon-glow-pha-bha-pore-tight-toner-P458219?icid2=products grid:p458219
https://www.sephora.com/product/glowmotions-P442821?icid2=products grid:p442821
https://www.sephora.com/product/glycol-lactic-radiance-renewal-mask-P203616?icid2=products grid:p203616
https://www.sephora.com/product/glycolic-acid-toner-P443838?icid2=products grid:p443838
https://www.sephora.com/product/good-genes-all-in-one-lactic-acid-treatment-P309308?icid2=products grid:p309308
https://www.sephora.com/product/grape-water-P12045?icid2=products grid:p12045
https://www.sephora.com/product/gravitymud-firming-treatment-P403486?icid2=products grid:p403486
https://www.sephora.com/product/hello-fab-coconut-skin-smoothie-priming-moisturizer-P415237?icid2=products grid:p415237
https://www.sephora.com/product/honey-potion-renewing-antioxidant-hydration-mask-with-echinacea-greenenvy-P410873?icid2=products grid:p410873
https://www.sephora.com/product/hyaluronic-acid-serum-P443845?icid2=products grid:p443845
https://www.sephora.com/product/instant-detox-mask-P395615?icid2=products grid:p395615
https://www.sephora.com/product/intensive-moisture-balance-P443358?icid2=products grid:p443358
https://www.sephora.com/product/invisiscar-resurfacing-treatment-P446423?icid2=products grid:p446423
https://www.sephora.com/product/juno-hydroactive-cellular-face-oil-P309306?icid2=products grid:p309306
https://www.sephora.com/product/kale-spinach-green-tea-age-prevention-cleanser-P411387?icid2=products grid:p411387
https://www.sephora.com/product/lapis-oil-balancing-facial-oil-P400203?icid2=products grid:p400203
https://www.sephora.com/product/lippe-tm-P403818?icid2=products grid:p403818
https://www.sephora.com/product/liquid-gold-with-glycolic-acid-P440949?icid2=products grid:p440949
https://www.sephora.com/product/lotus-youth-preserve-dream-face-cream-P440312?icid2=products grid:p440312
https://www.sephora.com/product/lotus-youth-preserve-rescue-mask-P426080?icid2=products grid:p426080
https://www.sephora.com/product/luminizing-black-mask-P266800?icid2=products grid:p266800
https://www.sephora.com/product/luminous-deep-hydration-firming-serum-P399624?icid2=products grid:p399624
https://www.sephora.com/product/luminous-dewy-skin-mist-P399623?icid2=products grid:p399623
https://www.sephora.com/product/luminous-dewy-skin-night-concentrate-P403820?icid2=products grid:p403820
https://www.sephora.com/product/luna-2-for-normal-skin-P404209?icid2=products grid:p404209
https://www.sephora.com/product/luna-2-for-sensitive-skin-P404210?icid2=products grid:p404210
https://www.sephora.com/product/luna-mini-2-P404444?icid2=products grid:p404444
https://www.sephora.com/product/luna-sleeping-night-oil-P393718?icid2=products grid:p393718
https://www.sephora.com/product/maracuja-c-brighter-eye-treatment-P377545?icid2=products grid:p377545
https://www.sephora.com/product/max-complexion-correction-pads-P114902?icid2=products grid:p114902
https://www.sephora.com/product/micro-essence-skin-activating-treatment-lotion-P387619?icid2=products grid:p387619
https://www.sephora.com/product/microdermabrasion-age-defying-exfoliator-P45503?icid2=products grid:p45503
https://www.sephora.com/product/midnight-recovery-concentrate-P421998?icid2=products grid:p421998
https://www.sephora.com/product/mini-facial-toning-device-P387138?icid2=products grid:p387138
https://www.sephora.com/product/moisture-bound-refreshing-hydra-gel-oil-free-P232940?icid2=products grid:p232940
https://www.sephora.com/product/moisture-surge-72-hour-auto-replenishing-hydrator-P425416?icid2=products grid:p425416
https://www.sephora.com/product/moisture-surge-cc-cream-hydrating-colour-corrector-broad-spectrum-spf-30-P378639?skuId=1509512&icid2=products grid:p378639
https://www.sephora.com/product/moisture-surge-overnight-mask-P377577?icid2=products grid:p377577
https://www.sephora.com/product/moisturizing-eye-bomb-P422905?icid2=products grid:p422905
https://www.sephora.com/product/multi-active-toner-P423150?icid2=products grid:p423150
https://www.sephora.com/product/multivitamin-power-recovery-masque-P423130?icid2=products grid:p423130
https://www.sephora.com/product/out-of-trouble-10-minute-mask-to-rescue-problem-skin-P297551?icid2=products grid:p297551
https://www.sephora.com/product/peace-out-microneedling-anti-wrinkle-retinol-patches-P453819?icid2=products grid:p453819
https://www.sephora.com/product/peace-out-pores-P428250?icid2=products grid:p428250
https://www.sephora.com/product/peace-out-salicylic-acid-acne-treatment-serum-P460509?icid2=products grid:p460509
https://www.sephora.com/product/peat-miracle-revital-cream-P412440?icid2=products grid:p412440
https://www.sephora.com/product/pep-start-2-in-1-exfoliating-cleanser-P409101?icid2=products grid:p409101
https://www.sephora.com/product/personal-microderm-P398500?icid2=products grid:p398500
https://www.sephora.com/product/personal-microderm-pro-P398502?icid2=products grid:p398502
https://www.sephora.com/product/phoenix-cell-regenerating-facial-oil-P400204?icid2=products grid:p400204
https://www.sephora.com/product/photo-finish-color-correcting-foundation-primer-P78811?icid2=products grid:p78811
https://www.sephora.com/product/photo-finish-foundation-primer-P9889?icid2=products grid:p9889
https://www.sephora.com/product/photo-finish-oil-free-foundation-primer-pore-minimizing-P387425?icid2=products grid:p387425
https://www.sephora.com/product/pineapple-c-brightening-serum-P442869?icid2=products grid:p442869
https://www.sephora.com/product/pitera-first-experience-kit-P447773?icid2=products grid:p447773
https://www.sephora.com/product/pmd-personal-microderm-with-hand-foot-kit-P401158?icid2=products grid:p401158
https://www.sephora.com/product/polyglutamic-acid-P443830?icid2=products grid:p443830
https://www.sephora.com/product/pores-no-more-pore-refiner-primer-P139005?icid2=products grid:p139005
https://www.sephora.com/product/potent-c-power-serum-P428100?icid2=products grid:p428100
https://www.sephora.com/product/powerful-strength-line-reducing-concentrate-12-5-vitamin-c-P427529?icid2=products grid:p427529
https://www.sephora.com/product/precleanse-P423148?icid2=products grid:p423148
https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421?icid2=products grid:p427421
https://www.sephora.com/product/pumpkin-enzymatic-dermal-resurfacer-P387261?icid2=products grid:p387261
https://www.sephora.com/product/pure-argan-milk-tm-intensive-hydrating-treatment-P384780?icid2=products grid:p384780
https://www.sephora.com/product/pure-maracuja-oil-P376726?icid2=products grid:p376726
https://www.sephora.com/product/purity-made-simple-pore-extractor-mask-P417621?icid2=products grid:p417621
https://www.sephora.com/product/r-n-a-power-radical-new-age-cream-P405950?icid2=products grid:p405950
https://www.sephora.com/product/rapid-relief-acne-spot-treatment-P421432?icid2=products grid:p421432
https://www.sephora.com/product/rare-earth-deep-pore-cleansing-mask-P422002?icid2=products grid:p422002
https://www.sephora.com/product/renewed-hope-in-jar-refreshing-refining-moisturizer-P392232?icid2=products grid:p392232
https://www.sephora.com/product/resurfacing-mask-P379707?icid2=products grid:p379707
https://www.sephora.com/product/resveratrol-lift-eye-lifting-balm-P400610?icid2=products grid:p400610
https://www.sephora.com/product/resveratrol-lift-night-infusion-cream-P400609?icid2=products grid:p400609
https://www.sephora.com/product/retinol-fusion-pm-P251529?icid2=products grid:p251529
https://www.sephora.com/product/retinol-serum-P443842?icid2=products grid:p443842
https://www.sephora.com/product/retinol-youth-renewal-serum-P411365?icid2=products grid:p411365
https://www.sephora.com/product/revitalizing-supreme-global-anti-aging-creme-P384342?icid2=products grid:p384342
https://www.sephora.com/product/rose-deep-hydration-face-cream-P406686?icid2=products grid:p406686
https://www.sephora.com/product/rose-deep-hydration-facial-toner-P417936?icid2=products grid:p417936
https://www.sephora.com/product/rose-deep-hydration-moisturizer-mini-P440813?icid2=products grid:p440813
https://www.sephora.com/product/rose-deep-hydration-sleeping-mask-P442858?icid2=products grid:p442858
https://www.sephora.com/product/rose-deep-hydration-toner-mini-P436353?icid2=products grid:p436353
https://www.sephora.com/product/rose-face-mask-P12336?icid2=products grid:p12336
https://www.sephora.com/product/rose-face-mask-mini-P426821?icid2=products grid:p426821
https://www.sephora.com/product/rose-floral-toner-P39970?icid2=products grid:p39970
https://www.sephora.com/product/salicylic-acid-cleanser-P443833?icid2=products grid:p443833
https://www.sephora.com/product/seaberry-skin-nutrition-cleansing-oil-P405935?icid2=products grid:p405935
https://www.sephora.com/product/shaba-complex-eye-serum-P419223?icid2=products grid:p419223
https://www.sephora.com/product/shani-darden-retinol-reform-P455926?icid2=products grid:p455926
https://www.sephora.com/product/shani-darden-retinol-reform-mini-P455927?icid2=products grid:p455927
https://www.sephora.com/product/shani-darden-texture-reform-P455928?icid2=products grid:p455928
https://www.sephora.com/product/sheer-transformation-perfecting-moisturizer-P42342?icid2=products grid:p42342
https://www.sephora.com/product/skin-perfecting-lotion-blemish-prone-oily-skin-P4050?icid2=products grid:p4050
https://www.sephora.com/product/skin-smoothing-cream-P443359?icid2=products grid:p443359
https://www.sephora.com/product/slaai-makeup-melting-butter-cleanser-P442566?icid2=products grid:p442566
https://www.sephora.com/product/smart-night-custom-repair-moisturizer-dry-combination-P404015?icid2=products grid:p404015
https://www.sephora.com/product/soy-face-cleanser-P7880?icid2=products grid:p7880
https://www.sephora.com/product/special-cleansing-gel-P423136?icid2=products grid:p423136
https://www.sephora.com/product/squalane-antioxidant-cleansing-oil-P416560?icid2=products grid:p416560
https://www.sephora.com/product/squalane-glycolic-renewal-facial-P433886?icid2=products grid:p433886
https://www.sephora.com/product/squalane-marine-algae-eye-cream-P447210?icid2=products grid:p447210
https://www.sephora.com/product/squalane-omega-repair-cream-P433887?icid2=products grid:p433887
https://www.sephora.com/product/squalane-peptide-eye-gel-P416562?icid2=products grid:p416562
https://www.sephora.com/product/squalane-phyto-retinol-serum-P429028?icid2=products grid:p429028
https://www.sephora.com/product/squalane-rose-vegan-lip-balm-P448210?icid2=products grid:p448210
https://www.sephora.com/product/squalane-tea-tree-balancing-oil-P442551?icid2=products grid:p442551
https://www.sephora.com/product/squalane-tea-tree-cleansing-gel-P442549?icid2=products grid:p442549
https://www.sephora.com/product/squalane-vitamin-c-rose-oil-P416563?icid2=products grid:p416563
https://www.sephora.com/product/sugar-face-polish-P12295?skuId=562082&icid2=products grid:p12295
https://www.sephora.com/product/sunday-riley-go-to-bed-with-me-evening-routine-kit-P463966?icid2=products grid:p463966
https://www.sephora.com/product/super-mud-clearing-treatment-P376409?icid2=products grid:p376409
https://www.sephora.com/product/super-spot-remover-acne-treatment-gel-P297517?icid2=products grid:p297517
https://www.sephora.com/product/superberry-hydrate-glow-dream-mask-P440307?icid2=products grid:p440307
https://www.sephora.com/product/supergoop-play-everyday-lotion-spf-50-with-sunflower-extract-P454383?icid2=products grid:p454383
https://www.sephora.com/product/superscreen-daily-moisturizer-broad-spectrum-spf-40-pa-P439928?icid2=products grid:p439928
https://www.sephora.com/product/supertoner-exfoliating-acid-solution-P441865?icid2=products grid:p441865
https://www.sephora.com/product/t-l-c-framboos-tm-glycolic-night-serum-P392246?icid2=products grid:p392246
https://www.sephora.com/product/t-l-c-sukari-babyfacial-P415777?icid2=products grid:p415777
https://www.sephora.com/product/tatcha-the-liquid-silk-canvas-featherweight-protective-primer-P456446?icid2=products grid:p456446
https://www.sephora.com/product/the-dewy-skin-cream-P441101?icid2=products grid:p441101
https://www.sephora.com/product/the-method-polish-P399737?icid2=products grid:p399737
https://www.sephora.com/product/the-microdelivery-exfoliating-wash-P184109?icid2=products grid:p184109
https://www.sephora.com/product/the-microdelivery-resurfacing-peel-P72504?icid2=products grid:p72504
https://www.sephora.com/product/the-nightly-retinol-peptide-anti-aging-serum-P438640?icid2=products grid:p438640
https://www.sephora.com/product/the-ordinary-deciem-ascorbyl-tetraisopalmitate-solution-20-in-vitamin-f-P427404?icid2=products grid:p427404
https://www.sephora.com/product/the-ordinary-deciem-azelaic-acid-suspension-10-P427411?icid2=products grid:p427411
https://www.sephora.com/product/the-ordinary-deciem-buffet-P427420?icid2=products grid:p427420
https://www.sephora.com/product/the-ordinary-deciem-glycolic-acid-7-toning-solution-P427406?icid2=products grid:p427406
https://www.sephora.com/product/the-ordinary-deciem-granactive-retinoid-2-emulsion-P427418?icid2=products grid:p427418
https://www.sephora.com/product/the-ordinary-deciem-hyaluronic-acid-2-b5-P427419?icid2=products grid:p427419
https://www.sephora.com/product/the-ordinary-deciem-lactic-acid-10-ha-2-P427413?icid2=products grid:p427413
https://www.sephora.com/product/the-ordinary-deciem-matrixyl-10-ha-P427405?icid2=products grid:p427405
https://www.sephora.com/product/the-ordinary-deciem-natural-moisturizing-factors-ha-P427414?icid2=products grid:p427414
https://www.sephora.com/product/the-ordinary-deciem-niacinamide-10-zinc-1-P427417?icid2=products grid:p427417
https://www.sephora.com/product/the-ordinary-deciem-vitamin-c-suspension-23-ha-spheres-2-P427410?icid2=products grid:p427410
https://www.sephora.com/product/the-ordinary-deciem-vitamin-c-suspension-30-in-silicone-P427409?icid2=products grid:p427409
https://www.sephora.com/product/the-silk-canvas-P428661?icid2=products grid:p428661
https://www.sephora.com/product/the-true-cream-aqua-bomb-P394639?icid2=products grid:p394639
https://www.sephora.com/product/the-true-cream-moisturizing-bomb-P394624?icid2=products grid:p394624
https://www.sephora.com/product/the-water-cream-P418218?icid2=products grid:p418218
https://www.sephora.com/product/therapeutic-sulfur-masque-acne-treatment-masque-P3617?icid2=products grid:p3617
https://www.sephora.com/product/thirstymud-tm-hydrating-treatment-P384782?icid2=products grid:p384782
https://www.sephora.com/product/treatment-enzyme-peel-P232931?icid2=products grid:p232931
https://www.sephora.com/product/trinity-eye-lip-enhancer-attachment-bundle-P385320?icid2=products grid:p385320
https://www.sephora.com/product/trinity-facial-toning-device-P383183?icid2=products grid:p383183
https://www.sephora.com/product/truth-on-the-glow-cleansing-cloths-P385342?icid2=products grid:p385342
https://www.sephora.com/product/truth-serum-P42343?icid2=products grid:p42343
https://www.sephora.com/product/turnaround-overnight-revitalizing-moisturizer-P296834?icid2=products grid:p296834
https://www.sephora.com/product/u-f-o-ultra-clarifying-face-oil-P410400?icid2=products grid:p410400
https://www.sephora.com/product/ultra-facial-cream-P421996?icid2=products grid:p421996
https://www.sephora.com/product/ultra-glow-serum-glow-strengthen-vitamin-c-serum-P442594?icid2=products grid:p442594
https://www.sephora.com/product/ultra-repair-cream-intense-hydration-P248407?skuId=1784826&icid2=products grid:p248407
https://www.sephora.com/product/ultra-repair-face-moisturizer-P375534?icid2=products grid:p375534
https://www.sephora.com/product/ultra-repair-hydrating-serum-P403739?icid2=products grid:p403739
https://www.sephora.com/product/ultra-repair-tinted-moisturizer-broad-spectrum-spf-30-P428652?icid2=products grid:p428652
https://www.sephora.com/product/umbra-tm-sheer-physical-daily-defense-broad-spectrum-sunscreen-spf-30-P419222?icid2=products grid:p419222
https://www.sephora.com/product/umbrian-clay-mattifying-face-exfoliant-P379065?icid2=products grid:p379065
https://www.sephora.com/product/umbrian-clay-purifying-mask-P54104?icid2=products grid:p54104
https://www.sephora.com/product/umbrian-clay-purifying-mask-mini-P426822?icid2=products grid:p426822
https://www.sephora.com/product/vinoperfect-radiance-serum-P94421?icid2=products grid:p94421
https://www.sephora.com/product/vinopure-natural-salicylic-acid-pore-minimizing-serum-P433626?icid2=products grid:p433626
https://www.sephora.com/product/vinopure-natural-salicylic-acid-pore-minimizing-toner-P433625?icid2=products grid:p433625
https://www.sephora.com/product/vinosource-moisturizing-sorbet-P271704?icid2=products grid:p271704
https://www.sephora.com/product/vinosource-s-o-s-thirst-quenching-serum-P374211?icid2=products grid:p374211
https://www.sephora.com/product/vinosource-sos-deep-hydration-moisturizer-P442852?icid2=products grid:p442852
https://www.sephora.com/product/virgin-marula-tm-luxury-facial-oil-P392245?icid2=products grid:p392245
https://www.sephora.com/product/vitamin-c-ester-brightening-serum-P378714?icid2=products grid:p378714
https://www.sephora.com/product/vitamin-nectar-moisture-glow-face-cream-P420158?icid2=products grid:p420158
https://www.sephora.com/product/waso-color-smart-day-moisturizer-broad-spectrum-spf-30-P421770?icid2=products grid:p421770
https://www.sephora.com/product/waso-color-smart-day-moisturizer-oil-free-broad-spectrum-spf-30-P421769?icid2=products grid:p421769
https://www.sephora.com/product/water-drench-hyaluronic-cloud-serum-P222818?icid2=products grid:p222818
https://www.sephora.com/product/watermelon-glow-sleeping-mask-P420160?icid2=products grid:p420160
https://www.sephora.com/product/watermelon-glow-ultra-fine-mist-P442744?icid2=products grid:p442744
https://www.sephora.com/product/watermelon-pink-juice-moisturizer-P428819?icid2=products grid:p428819
https://www.sephora.com/product/whipped-argan-oil-face-butter-P404338?icid2=products grid:p404338
https://www.sephora.com/product/youth-to-the-people-yerba-mate-fruit-enzyme-high-caffeine-resurfacing-facial-P455934?icid2=products grid:p455934'''
# cleaning this into usable list of urls
this_list = my_str.split(" ")
condensed_seph_links = []
condensed_seph_links.append(this_list[0])
for group in this_list[1:]:
  if ("\nhttps://" in group):
    condensed_seph_links.append("https://" + group.split("\nhttps://")[1])
print(condensed_seph_links)



In [ ]:
# iterating through url list to scrape from all pages
seph_products = []
for product in condensed_seph_links:
  curr_index = requests.get(product, auth=('user', 'pass'))
  curr_soup = BeautifulSoup(curr_index.content)
  curr_list = parse_store(curr_soup, product)
  for item in curr_list:
    seph_products.append(item)
print(seph_products)

transferring to JSON to transfer to pandas dataframe

In [ ]:
# dumping from list to JSON
with open('sephora.json', 'w') as fout:
  json.dump(products, fout)

In [ ]:
# reading in from JSON
with open('sephora.json', 'r') as fd:
  seph_products = json.load(fd)
print(seph_products)

In [ ]:
# changing to pandas dataframe
sephora_df = pd.DataFrame(products)
sephora_df = sephora_df.dropna()
sephora_df

## Cleaning

Now that we have gathered all of the data we wanted to scrape into three main pandas dataframes
- byrdie_df
- the_ordinary_df
- sephora_df
We want to create the tables that will link these dataframes together to provide a diagnosis.

Thus, we will need two additional tables
1. Linking symptoms to chemicals
2. Linking chemicals to products



### Symptoms
I was originally intended to try and use some convolutional neural nets or some kind of deep learning technique to have the computer decide for itself how to identify key words/features associated with each chemical and what words are related to what symptom. For the sake of finishing this project on time, however, I decided to just set my own "rules."

I narrowed down the options people could choose for skincare recommendations into 13 options.

In [ ]:
# symptom options formatted in a list
symptoms = ["uneven skintone", "acne", "dullness", "dryness", "redness", "aging", "sun protectant", "wound", "itchiness", "oiliness", "exfoliator/cleanser", "rough", "discomfort"]

I then went through my dataframe of chemicals and identified all the different terms that could be associated with what one may want from a skincare product.

In [ ]:
# this is a set of all of the terms in all their different permutations that I gathered from byrdie_df
# to get rid of duplicates I used the set function from python and then reformatted it ias a list, chem_descriptors
myset = set(["moisturizes", "soothes", "itchiness", "injured", "calms", "hydrates"", exfoliate", "dullness", "even", "texture", "antioxidant", "hydrating", "moisturize", "infection", "balance", "pH", "exfoliates", "antibacterial", "lightens", "dark spots", "scars", "tone", "wrinkle", "fine lines", "moisture", "pigment", "fade", "collagen", "hydration", "damage", "rosacea", "acne", "lighten", "removes", "dead", "firmness", "pores", "bacteria", "acne-causing", "preserves", "stabilizes", "boosts", "smooths", "antioxidant", "overall", "retains", "brightens", "inflammation", "spoiling", "absorb", "turnover", "detox", "detoxifying", "soothing", "rebuild", "restore", "aging", "environmental","thickens", "toxins", "heals", "chapped", "water loss", "evens", "sun damage", "silky", "seals", "cracks", "lubricating", "antimicrobial", "anti-inflammatory", "spots", "texture", "dewy", "hyperpigmentation", "circles", "scars", "tightens", "sun", "sebum", "oil", "replenishes", "wound", "redness", "greasy", "discoloration", "firmer", "lesions", "moisturize", "fungal", "melanin", "sensitive", "brighter", "unclogged", "anti-acne", "anti-aging", "eczema", "psoriasis", "softens", "barrier", "fine", "line", "lines", "cleanses", "cleanse", "brightening", "cancer","soft", "supple", "repairs", "regenerate", "detoxes", "UV", "barrier-boosting", "water", "moisture", "unclog", "dermis", "thicken", "elasticity", "oil production", "dull", "dryness", "clean", "hormonal", "androgen", "burn", "free radical"])
chem_descriptors = list(myset)
chem_descriptors

For each of the descriptors in chem_descriptors, I went through by hand and assigned them a key corresponding to the symptom that descriptor is associated with.

In [ ]:
# dictionary of associations of chemical descriptors to the symptoms they could treat/describe
chem_descriptors_to_symp = {'UV': "sun protectant", "stabilizes" : "discomfort", "antioxidant": "dullness", 
                            "spoiling" : "discomfort", "seals": "dryness", "chapped" : "dryness", "discoloration" : "uneven skintone",
                            "smooths": "rough", "clean" : "exfoliator/cleanser", "dead" : "aging", "damage" : "sun protectant",
                            "boosts" : "dullness", "even" : "uneven skintone", "brighter" : "dullness", "rosacea" : "discomfort", "lubricating" : "dryness", 
                            "circles" : "uneven skintone", "exfoliates" : "exfoliator/cleanser", "toxins" : "exfoliator/cleanser", "restore" : "dryness", 
                            "moisture" : "dryness", "cleanses" : "exfoliator/cleanser", "aging" : "aging", "removes" : "exfoliator/cleanser", 
                            "brightens": "dullness", "antimicrobial" : "acne", "sun damage" : "sun protectant",
                            "anti-inflammatory" : "redness", "line" : "aging", "water" : "dryness", "heals" : "dryness", 
                            "repairs" : "discomfort", "dermis" : "dryness", "dewy" : "dryness", "antibacterial" : "acne", 
                            "hormonal" : "acne", "detoxifying" : "exfoliator/cleanser", "dark spots" : "uneven skintone", 
                            "thickens" : "discomfort", "environmental" :  "sun protectant", "barrier" : "dryness", 
                            "itchiness" : "discomfort", "hydrates" : "dryness", "exfoliate" : "exfoliator/cleanser", 
                            "dryness" : "dryness", "inflammation" : "redness", "infection" : "acne", "bacteria" : "acne", 
                            "pigment" : "uneven skintone", "sebum" : "acne", "scars": "acne", "spots": "uneven skintone", 
                            "unclogged" : "acne", "lesions" : "acne", "acne" : "acne", "oil production" : "oiliness",
                            "pores": "acne", "replenished": "dryness", "injured" : "discomfort", "retains" : "dryness", 
                            "moisturize" : "dryness", "anti-acne" : "acne", "wound" : "discomfort", "softens" : "rough", 
                            "fungal" : "discomfort", "firmer" : "aging", "sensitive" : "discomfort", "sun" : "sun protectant", 
                            "texture" : "rough", "tone" : "uneven skintone", "soothing" : "discomfort", "melanin" : "uneven skintone",
                            "thicken" : "dullness", "overall" : "discomfort", "androgen" : "acne", "hydrating" : "dryness", 
                            "water loss" : "hydrating", "absorb" : "hydrating", "evens" : "uneven skintone", "soothes" : "discomfort", 
                            "fade" : "aging", "cleanse": "exfoliator/cleanser", "calms" : "discomfort", "dull" : "dullness", 
                            "acne-causing" : "acne", "fine lines" : "aging", "supple" : "rough", "soft" : "rough", "fine" : "aging", 
                            "regenerate" : "dryness", "free radical" : "sun protectant", "hyperpigmentation" : "uneven skintone", 
                            "oil" : "oiliness", "firmness" : "aging", "preserves" : "aging", "eczema" : "discomfort", 
                            "lines" : "aging", "detoxes" : "exfoliator/cleanser", "turnover" : "dullness", "balance" : "discomfort", 
                            "pH" : "dullness", "detox" : "exfoliator/cleanser", "burn" : "discomfort", "cancer" : "sun-protectant", 
                            "anti-aging" : "aging", "rebuild" : "discomfort", "barrier-boosting" : "dryness", "cracks" : "dryness", 
                            "greasy" : "oiliness", "wrinkle" : "aging", "silky" : "rough", "unclog" : "acne", "tightens" : "aging", 
                            "redness" : "redness", "elasticity" : "aging", "lighten" : "uneven skintone", "brightening" : "dullness"}
chem_descriptors_to_symp

### Connecting Symptoms to Chemicals

Now, I wanted to create a dataframe that would contain rows of just a symptom and a chemical whose description in "benefits" section of byrdie_df contained chemical descriptors that were associated with that symptom as determined by our chem_descriptors_to_symp dictionary.

In [ ]:
# helper function to identify the dictionary values found in a given text 
# and return a list of the symptoms identified from this text
def id_properties(text, this_dict):
  symptoms = []
  for term in this_dict.keys():
    if term in text.lower():
      if (this_dict[term] not in symptoms):
        symptoms.append(this_dict[term])
  return symptoms

In [ ]:
## going through our byrdie_df chemicals database to create a new dataframe with chemicals and associated symptoms
chem_names = []
chem_symptoms = []
counter = 0
for name in byrdie_df['name']:
   curr_name = name
   text = byrdie_df.iloc[counter, 2]
   properties = id_properties(text, chem_descriptors_to_symp)
   for thing in properties:
     chem_names.append(curr_name)
     chem_symptoms.append(thing)
   counter += 1
chem_symptoms_dict = {'chem_name': chem_names, 'chem_symptoms':chem_symptoms} 
chem_to_symp_df = pd.DataFrame(chem_symptoms_dict, columns = ['chem_name', 'chem_symptoms'])
chem_to_symp_df = chem_to_symp_df.dropna()
chem_to_symp_df

### Connecting Chemicals to Products

Fortunately, the products don't really have to be cleaned since in both Sephora and the Ordinary, the ingredients are already in the right format, just a string of all of the ingredients.

In [ ]:
# helper function to identify the ingredients from a text given a list of ingredients to search for
def id_ingredients(text, this_list):
  chemicals = []
  for term in this_list:
    if term == "Vitamin C":
      if "Ascorbic Acid" in text and term not in chemicals:
        chemicals.append(term)
    if term in text and term not in chemicals:
      chemicals.append(term)
  return chemicals

In [ ]:
# gathering all chemical names in a list
all_chem_names = [name for name in byrdie_df['name'] if str(name) != 'nan']

I am only showing how to connect chemicals to sephora products because that was what was ultimately used, but it is essentially the same idea for The Ordinary except the index for "iloc" should be the index of where the ingredients are in the_ordinary_df

In [ ]:
# working through all of the products in sephora_df to create new dataframe of those products and their chemicals
counter = 0
chem_names_1 = []
prod_names = []
for name in sephora_df['name']:
   curr_name = name
   text = sephora_df.iloc[counter, 5]
   properties = id_ingredients(text, all_chem_names)
   for thing in properties:
     prod_names.append(curr_name)
     chem_names_1.append(thing)
   counter += 1

chem_prod_dict = {'chem_name': chem_names_1, 'prod_name':prod_names} 
chem_to_prod_df = pd.DataFrame(chem_prod_dict, columns = ['chem_name', 'prod_name'])
chem_to_prod_df


### Converting to CSV
For ease of use, I decided not to use SQLAlchemy to convert the dataframes directly to SQLite, but to instead convert to csv and then read in row by row into a SQLite database in the IDE

In [ ]:
# converting byrdie_df to csv
byrdie_df.to_csv("chem_full.csv", encoding = "utf-8", index = False)
# converting chem_to_symp_df to csv
chem_to_symp_df.to_csv("symp_to_chem_names.csv", encoding = "utf-8", index = False)
# converting all ordinary products to csv
the_ordinary_df.to_csv("prod_ordinary_full.csv", encoding = "utf-8", index = False)
# converting all sephora products to csv
sephora_df.to_csv("prod_sephora_full.csv", encoding = "utf-8", index = False)
# converting chem_to_prod_df to csv
chem_to_prod_df.to_csv("chem_to_sephora_prod.csv", encoding = "utf-8", index = False)

# Final Data Directory
See all of the major lists, dictionaries, dataframes scraped + cleaned


In [ ]:
# dataframe of all chemicals
byrdie_df

# list of condensed symptoms
symptoms

# dictionary for translating natural language to condensed symptoms
chem_descriptors_to_symp

# dataframe connecting symptoms to chemicals
chem_to_symp_df

# dataframe of all the ordinary products
the_ordinary_df

#dataframe of all the sephora products 
sephora_df

# list of all chemical names
all_chem_names

# dataframe connecting chemicals to sephora products
chem_to_prod_df


# Final Notes
If you have any questions, please feel free to reach out!

**Email**: cindywang@college.harvard.edu

**Mobile**: 845-743-6807
